In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Accessing the CIFAR10 dataset

In [14]:
(ds_train,ds_test), ds_info = tfds.load(
    "cifar10",
    split = ['train','test'],
    shuffle_files= True,
    as_supervised = True, #tuple of img,label
    with_info = True
)

### Preprocessing & Model Creation

In [4]:
def normalize_img(image,label):
    #normalize images
    return tf.cast(image, tf.float32)/255.0 , label

AUTOTUNE = tf.data.experimental.AUTOTUNE #to autotune hyperparameters where it is called

In [ ]:
#setup training dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls = AUTOTUNE) #applies function to call images, labels
ds_train = ds_train.cache() 
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(AUTOTUNE)

In [ ]:
#setup testing dataset
ds_test = ds_test.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

In [5]:
model = keras.Sequential(
    [
        keras.Input((32,32,3)),
        layers.Conv2D(4,3, padding = 'same', activation = 'relu'),
        layers.Conv2D(8,3, padding = 'same', activation = 'relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding='same', activation = 'relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation = 'softmax')
    ]
)

In [6]:
model.compile(
    optimizer = keras.optimizers.Adam(3e-4),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

### Performing Data Augmentation
There will be covered 2 methods:
1. In case we are working with tfds or tf.data api (custom datasets)
2. 

#### TFDS or Tf.Data

In [12]:
def augment(image, label):
    new_height = new_width = 32
    
    #resize
    image = tf.image.resize(image, (new_height,new_width))
    
    if tf.random.uniform((),minval=0,maxval=1) < 0.1:
        #10% probability will give this condition true, randomly
        image = tf.tile(tf.image.rgb_to_grayscale(image),[1,1,3]) #tile is used to make 1 channel into 3
    
    image = tf.image.random_brightness(image, max_delta = 0.1) #as the name says
    image = tf.image.random_contrast(image, lower=0.1, upper = 0.2)
    
    image = tf.image.random_flip_left_right(image) #50% times
#     image = tf.image.random_flip_up_down(image) #50% times
    
    #and other such functions can be used. 
    
    return image, label

In [15]:
#setup training dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls = AUTOTUNE) #applies function to call images, labels
ds_train = ds_train.cache() 
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE) #applies augmentation function
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(AUTOTUNE)

#setup testing dataset
ds_test = ds_test.map(normalize_img, num_parallel_calls = AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

In [16]:
model.fit(ds_train, epochs=30, verbose=2)

Epoch 1/30
782/782 - 2s - loss: 2.2302 - accuracy: 0.2083
Epoch 2/30
782/782 - 2s - loss: 2.1944 - accuracy: 0.2542
Epoch 3/30
782/782 - 2s - loss: 2.1721 - accuracy: 0.2779
Epoch 4/30
782/782 - 2s - loss: 2.1570 - accuracy: 0.2954
Epoch 5/30
782/782 - 2s - loss: 2.1448 - accuracy: 0.3092
Epoch 6/30
782/782 - 2s - loss: 2.1324 - accuracy: 0.3216
Epoch 7/30
782/782 - 2s - loss: 2.1235 - accuracy: 0.3312
Epoch 8/30
782/782 - 2s - loss: 2.1148 - accuracy: 0.3418
Epoch 9/30
782/782 - 2s - loss: 2.1053 - accuracy: 0.3515
Epoch 10/30
782/782 - 2s - loss: 2.0982 - accuracy: 0.3594
Epoch 11/30
782/782 - 2s - loss: 2.0923 - accuracy: 0.3647
Epoch 12/30
782/782 - 2s - loss: 2.0859 - accuracy: 0.3705
Epoch 13/30
782/782 - 2s - loss: 2.0813 - accuracy: 0.3766
Epoch 14/30
782/782 - 2s - loss: 2.0747 - accuracy: 0.3826
Epoch 15/30
782/782 - 2s - loss: 2.0690 - accuracy: 0.3872
Epoch 16/30
782/782 - 2s - loss: 2.0644 - accuracy: 0.3913
Epoch 17/30
782/782 - 2s - loss: 2.0602 - accuracy: 0.3958
Epoch 

#### Second Option
Including Data Augmentation in the model itself so that it becomes easy

In [18]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=32,width=32),
        layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
        layers.experimental.preprocessing.RandomContrast(factor=0.1)
    ]
)

In [19]:
model = keras.Sequential(
    [
        keras.Input((32,32,3)),
        data_augmentation,
        layers.Conv2D(4,3, padding = 'same', activation = 'relu'),
        layers.Conv2D(8,3, padding = 'same', activation = 'relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding='same', activation = 'relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation = 'softmax')
    ]
)

In [20]:
model.compile(
    optimizer = keras.optimizers.Adam(3e-4),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [21]:
model.fit(ds_train, epochs=10, verbose=2)

Epoch 1/10
782/782 - 3s - loss: 2.2494 - accuracy: 0.1833
Epoch 2/10
782/782 - 2s - loss: 2.2177 - accuracy: 0.2263
Epoch 3/10
782/782 - 2s - loss: 2.1922 - accuracy: 0.2579
Epoch 4/10
782/782 - 2s - loss: 2.1716 - accuracy: 0.2795
Epoch 5/10
782/782 - 2s - loss: 2.1600 - accuracy: 0.2925
Epoch 6/10
782/782 - 2s - loss: 2.1490 - accuracy: 0.3049
Epoch 7/10
782/782 - 2s - loss: 2.1376 - accuracy: 0.3159
Epoch 8/10
782/782 - 2s - loss: 2.1290 - accuracy: 0.3271
Epoch 9/10
782/782 - 2s - loss: 2.1239 - accuracy: 0.3313
Epoch 10/10
782/782 - 2s - loss: 2.1161 - accuracy: 0.3401
